---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    #%matplotlib notebook
    #import matplotlib.pyplot as plt
    # Your Code Here
    #G1 = P1_Graphs[0]
    #G2 = P1_Graphs[1]
    #G3 = P1_Graphs[2]
    #G4 = P1_Graphs[3]
    #G5 = P1_Graphs[4]

    #g1_degrees = G1.degree()
    #g2_degrees = G2.degree()
    #g3_degrees = G3.degree()
    #g4_degrees = G4.degree()
    #g5_degrees = G5.degree()

    #fig, ((ax1,ax2,ax3), (ax4,ax5,ax6)) = plt.subplots(2, 3, sharey=True)
    #ax1.hist(list(g1_degrees.values())) ##Power Law
    #ax2.hist(list(g2_degrees.values()))
    #ax3.hist(list(g3_degrees.values()))
    #ax4.hist(list(g4_degrees.values())) ##Power Law
    #ax5.hist(list(g5_degrees.values()))

    #clust1 = nx.average_clustering(G1) 
    #clust2 = nx.average_clustering(G2)  ##High clustering Coefs
    #clust3 = nx.average_clustering(G3)  ##High clustering Coefs
    #clust4 = nx.average_clustering(G4)
    #clust5 = nx.average_clustering(G5)  ##low clustering Coefs

    #average_path1 = nx.average_shortest_path_length(G1)
    #average_path2 = nx.average_shortest_path_length(G2)
    #average_path3 = nx.average_shortest_path_length(G3)
    #average_path4 = nx.average_shortest_path_length(G4)
    #average_path5 = nx.average_shortest_path_length(G5)
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']# Your Answer Here
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    Evaluation = df.loc[df['ManagementSalary'].isnull()]
    data = df.loc[df['ManagementSalary'].notnull()]
    X_data = data[['Department', 'clustering', 'degree']]
    y_data = data['ManagementSalary']
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=0) #Used for tuning 
    y_scores = LogisticRegression(C=100).fit(X_data, y_data).decision_function(Evaluation[['Department', 'clustering', 'degree']])
    return pd.Series(y_scores, index= Evaluation.index)# Your Answer Here
salary_predictions()

1      -2.201833
2       0.564595
5       3.938056
8      -2.064239
14     -0.851180
18     -1.848276
27     -1.036683
30     -0.637410
31     -1.559795
34     -1.689415
37     -2.182134
40     -1.229022
45     -1.723357
54     -0.872144
55     -1.231318
60     -1.402759
62      6.703682
65     -0.432788
77     -2.948665
79     -2.363640
97     -2.237675
101    -2.967629
103    -1.557818
108    -2.362063
113    -0.055318
122    -3.285442
141    -0.346193
142     3.062624
144    -2.326907
145    -0.075323
          ...   
913    -3.082428
914    -2.434330
915    -5.783984
918    -1.921417
923    -2.944123
926    -2.190345
931    -2.780218
934    -3.902876
939    -4.460226
944    -5.575410
945    -2.997417
947    -1.461631
950    -2.241804
951    -2.916762
953    -3.713091
959    -5.905653
962    -5.765452
963    -1.930074
968    -1.642349
969    -1.748938
974    -2.401464
984    -3.956170
987    -1.864914
989    -1.764018
991    -1.346868
992    -5.799064
994    -5.610172
996    -5.7318

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
def new_connections_predictions():
    
    # Your Code Here
    future_connections['Common_neigh'] = future_connections.apply(lambda x: len(list(nx.common_neighbors(G,x.name[0],x.name[1]))), axis=1) 
    Evaluation = future_connections.loc[future_connections['Future Connection'].isnull()]
    data = future_connections.loc[future_connections['Future Connection'].notnull()]
    X_data = data[['Common_neigh']]
    y_data = data['Future Connection']
    y_scores = LogisticRegression(C=100).fit(X_data, y_data).decision_function(Evaluation[['Common_neigh']])
    return pd.Series(y_scores, index= Evaluation.index)# Your Answer Here
new_connections_predictions()

(107, 348)   -3.278414
(542, 751)   -4.281286
(20, 426)     0.733073
(50, 989)    -4.281286
(942, 986)   -4.281286
(324, 857)   -4.281286
(13, 710)    -1.272671
(19, 271)    -1.272671
(319, 878)   -4.281286
(659, 707)   -4.281286
(49, 843)    -4.281286
(208, 893)   -4.281286
(377, 469)   -4.281286
(405, 999)   -3.779850
(129, 740)   -3.779850
(292, 618)   -3.779850
(239, 689)   -4.281286
(359, 373)   -4.281286
(53, 523)    -3.278414
(276, 984)   -4.281286
(202, 997)   -4.281286
(604, 619)   -3.278414
(270, 911)   -4.281286
(261, 481)   -2.275542
(200, 450)    1.234509
(213, 634)   -4.281286
(644, 735)   -3.278414
(346, 553)   -4.281286
(521, 738)   -4.281286
(422, 953)   -3.779850
                ...   
(672, 848)   -4.281286
(28, 127)     2.738817
(202, 661)   -4.281286
(54, 195)     7.251740
(295, 864)   -4.281286
(814, 936)   -4.281286
(839, 874)   -4.281286
(139, 843)   -4.281286
(461, 544)   -4.281286
(68, 487)    -4.281286
(622, 932)   -4.281286
(504, 936)   -3.779850
(479, 528) 